In [1]:
pip install sounddevice scipy jiwer whisper vosk SpeechRecognition ffmpeg-python

  Obtaining dependency information for jiwer from https://files.pythonhosted.org/packages/ba/f4/35634d9eeff3b0bab51f5b9474ee569b1186bf29cf0d9d67b84acc80c53d/jiwer-3.1.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.8 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 42.8/42.8 kB 517.4 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for click>=8.1.8 from https://files.pythonhosted.org/packages/85/32/10bb5764d90a8eee674e9dc6f4db6a0ab47c8c4d0d83c27f7c39ac415a4d/click-8.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for rapidfuzz>=3.9.7 from https://files.pythonhosted.org/packages/c9/5a/d00e1f63564050a20279015acb29ecaf41646adfacc6ce2e1e450f7f2633/rapidfuzz-3.13.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1

In [3]:
# vosk_model_path 본인의 다운로드 경로로 수정
# ffmpeg를 다운로드 받아 시스템 path에 등록되어야함(ffmpeg -version으로 확인)

In [9]:
import sounddevice as sd
from scipy.io.wavfile import write
import whisper
from vosk import Model, KaldiRecognizer
import wave
import json
import speech_recognition as sr
import jiwer
import time
import os
import pandas as pd

# 📌 설정
fs = 16000
seconds = 5
ground_truth = "안녕하세요 저는 인공지능을 공부하는 학생입니다"
audio_path = "mic_test.wav"
pcm_path = "mic_test_pcm.wav"
vosk_model_path = "C:/vosk-model-small-ko-0.22"  # 본인 경로에 맞게 수정

# 🎙️ 1. 마이크 녹음
print("🎙️ 5초간 녹음 시작...")
recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
sd.wait()
write(audio_path, fs, recording)
print("✅ 녹음 완료")

results = []

# 🧠 2. Whisper
start = time.time()
model_whisper = whisper.load_model("base")
result_whisper = model_whisper.transcribe(audio_path)
whisper_text = result_whisper["text"].strip()
wer_whisper = jiwer.wer(ground_truth, whisper_text)
whisper_time = time.time() - start
results.append(("Whisper", round(whisper_time, 2), round(wer_whisper, 4), whisper_text))

# 🧠 3. Vosk
# PCM으로 변환
os.system(f'ffmpeg -y -i "{audio_path}" -acodec pcm_s16le -ar 16000 -ac 1 "{pcm_path}"')

start = time.time()
model_vosk = Model(vosk_model_path)
wf = wave.open(pcm_path, "rb")
rec = KaldiRecognizer(model_vosk, wf.getframerate())
vosk_text = ""
while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        result = json.loads(rec.Result())
        vosk_text += result.get("text", "") + " "
vosk_text = vosk_text.strip()
wer_vosk = jiwer.wer(ground_truth, vosk_text)
vosk_time = time.time() - start
results.append(("Vosk", round(vosk_time, 2), round(wer_vosk, 4), vosk_text))

# 🧠 4. Google
start = time.time()
recognizer = sr.Recognizer()
with sr.AudioFile(pcm_path) as source:
    audio_data = recognizer.record(source)
try:
    google_text = recognizer.recognize_google(audio_data, language="ko-KR").strip()
except Exception as e:
    google_text = f"Error: {e}"
wer_google = jiwer.wer(ground_truth, google_text) if "Error" not in google_text else None
google_time = time.time() - start
results.append(("Google", round(google_time, 2), round(wer_google, 4) if wer_google else None, google_text))

# 📊 5. 결과 출력
df = pd.DataFrame(results, columns=["모델", "실행 시간 (초)", "WER", "인식 결과"])
print(df)


🎙️ 5초간 녹음 시작...
✅ 녹음 완료


C:\Users\xw47PnC7Thon\anaconda3\Lib\site-packages\whisper\model.py:124: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  a = scaled_dot_product_attention(


        모델  실행 시간 (초)  WER           인식 결과
0  Whisper       2.47  1.0                
1     Vosk       2.71  1.0                
2   Google       1.33  1.0  가나다라마바사아자차카타파하


In [17]:
import sounddevice as sd
from scipy.io.wavfile import write
import whisper
from vosk import Model, KaldiRecognizer
import wave
import json
import speech_recognition as sr
import jiwer
import time
import os
import pandas as pd

# 📌 설정
fs = 16000
seconds = 5
ground_truth = "안녕하세요 저는 인공지능을 공부하는 학생입니다"
audio_path = "mic_test.wav"
pcm_path = "mic_test_pcm.wav"
vosk_model_path = "C:/vosk-model-small-ko-0.22"
  # 본인 경로에 맞게 수정

# 🎙️ 1. 마이크 녹음
print("🎙️ 5초간 녹음 시작...")
recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
sd.wait()
write(audio_path, fs, recording)
print("✅ 녹음 완료")

results = []

# 🧠 2. Whisper
start = time.time()
model_whisper = whisper.load_model("base")
result_whisper = model_whisper.transcribe(audio_path)
whisper_text = result_whisper["text"].strip()
wer_whisper = jiwer.wer(ground_truth, whisper_text)
whisper_time = time.time() - start
results.append(("Whisper", round(whisper_time, 2), round(wer_whisper, 4), whisper_text))

# 🧠 3. Vosk
# PCM으로 변환
os.system(f'ffmpeg -y -i "{audio_path}" -acodec pcm_s16le -ar 16000 -ac 1 "{pcm_path}"')

start = time.time()
model_vosk = Model(vosk_model_path)
wf = wave.open(pcm_path, "rb")
rec = KaldiRecognizer(model_vosk, wf.getframerate())
vosk_text = ""
while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        result = json.loads(rec.Result())
        vosk_text += result.get("text", "") + " "
vosk_text = vosk_text.strip()
wer_vosk = jiwer.wer(ground_truth, vosk_text)
vosk_time = time.time() - start
results.append(("Vosk", round(vosk_time, 2), round(wer_vosk, 4), vosk_text))

# 🧠 4. Google
start = time.time()
recognizer = sr.Recognizer()
with sr.AudioFile(pcm_path) as source:
    audio_data = recognizer.record(source)
try:
    google_text = recognizer.recognize_google(audio_data, language="ko-KR").strip()
except Exception as e:
    google_text = f"Error: {e}"
wer_google = jiwer.wer(ground_truth, google_text) if "Error" not in google_text else None
google_time = time.time() - start
results.append(("Google", round(google_time, 2), round(wer_google, 4) if wer_google else None, google_text))

# 📊 5. 결과 출력
df = pd.DataFrame(results, columns=["모델", "실행 시간 (초)", "WER", "인식 결과"])
print(df)


🎙️ 5초간 녹음 시작...
✅ 녹음 완료
        모델  실행 시간 (초)  WER                      인식 결과
0  Whisper       1.27  0.2   안녕하세요 저는 인공지능을 공부한 학생입니다
1     Vosk       3.00  1.0                           
2   Google       1.27  NaN  안녕하세요 저는 인공지능을 공부하는 학생입니다
